# Spatial join

[A Spatial join](http://wiki.gis.com/wiki/index.php/Spatial_Join) is yet
another classic GIS problem. Getting attributes from one layer and
transferring them into another layer based on their spatial relationship
is something you most likely need to do on a regular basis.

The previous materials focused on learning how to perform a 
[Point in Polygon query](point-in-polygon.html). We could now apply those
techniques and create our own function to perform a spatial join between
two layers based on their spatial relationship. We could for example
join the attributes of a polygon layer into a point layer where each
point would get the attributes of a polygon that `contains` the point.

Luckily, [spatial joins](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html)
(`gpd.sjoin()` -function) is already implemented in Geopandas, thus we
do not need to create it ourselves. There are three possible types of
join that can be applied in spatial join that are determined with `op`
-parameter:

-   `"intersects"`
-   `"within"`
-   `"contains"`

Sounds familiar? Yep, all of those spatial relationships were discussed
in the [previous materials](point-in-polygon.html), and even 
[Shapely basics](../Py_01/Geometric-Objects.html) thus you should know
how they work.

Let's perform a spatial join between the species monitoring data
GeoPackage file, [category_3_species_porijogi.gpkg](../files/data/L3/category_3_species_porijogi.gpkg)
and a Polygon layer that is extracted Corine Landuse Cover for the
Porijogi region.

## Download and clean the data

For this lesson we will be using publicly available Corine Landuse Cover for the Porijogi region in Tartumaa:

- Download [file:porijogi_corine_landuse.zip](../files/data/L3/porijogi_corine_landuse.zip)
- Unzip the file into into your L3 folder:

``` 
porijogi_corine_landuse.dbf  porijogi_corine_landuse.shp
porijogi_corine_landuse.prj  porijogi_corine_landuse.shx
```

You should now have the files listed above in your Data folder.

Let's read the data into geopandas and see what we have.

In [ ]:
import geopandas as gpd

# Filepath
fp = "../files/data/L3/porijogi_corine_landuse.shp"

# Read the data
lulc = gpd.read_file(fp)

# See the first rows
display(lulc.head(5))

We have multiple columns in the dataset but the most important
one here is the column `clc_int` (*corine landuse code*) that tells the
type of landuse cover under that polygon. Download the codes lookup
table:

- [file:corine_landuse_codes.csv](../files/data/L3/corine_landuse_codes.csv)

In order to know what the codes mean, we will merge a lookup table to
our spatial landuse cover GeoDataframe.

In [ ]:
import pandas as pd

codes = pd.read_csv('../files/data/L3/corine_landuse_codes.csv', sep=';')
display(codes.head(5))

This table contains a field `CLC_CODE` which we use to connect the
correct mapping to our landuse cover GeoDataframe, which has a field
`clc_int`. We will now merge the lookup table `codes` (a Pandas
dataframe) into our `lulc` GeoDataframe, based on the identifiers in the
mentioned fields:

In [ ]:
# general pandas table join
lulc = lulc.merge(codes, left_on='clc_int', right_on='CLC_CODE')
display(lulc.sample(5))

In [ ]:
# See the column names and confirm that we now have a column called
# LABEL2, that gives us some textual description for the landuse codes
display(lulc.columns)

Let's also get rid of the unnecessary columns by selecting only columns
that we need i.e. `Landuse`, `LABEL2` and `geometry`

In [ ]:
# Columns that will be selected
selected_cols = ['Landuse', 'LABEL2','geometry']

# Select those columns, we intentional assign to and overwrite the same variable
lulc = lulc[selected_cols]

# Let's see 5 randomly sampled rows
display(lulc.sample(5))

Now we have cleaned the data and have only those columns that we need
for our analysis.

## Join the layers

Now we are ready to perform the spatial join between the two layers that
we have. The aim here is to get information about **how many species
sightings (of which species) happened in which landuse types?** . Thus,
we want to join attributes from the landuse layer we just modified into
the already used and prepared monitoring GeoPackage file:

- [category_3_species_porijogi.gpkg](../files/data/L3/category_3_species_porijogi.gpkg)

Read the category_3_species_porijogi.gpkg layer into memory

In [ ]:
# protected species under class 3 monitoring sightings
species_fp = "../files/data/L3/category_3_species_porijogi.gpkg"

# Read data 
species = gpd.read_file(species_fp, layer='category_3_species_porijogi', driver='GPKG')

Let's make sure that the coordinate reference system of the layers are
identical:

In [ ]:
# Check the crs of landuse
print(lulc.crs)

# Check the crs of species layer in case we need to reproject the
# geometries to make them comparable
print(species.crs)

# Do they match? - We can test that
print(f"Do they match: {lulc.crs == species.crs} ")

They are identical. Thus, we can be sure that when doing spatial queries
between layers the locations match and we get the right results e.g.
from the spatial join that we are conducting here.

Let's now join the attributes from `lulc` (2) GeoDataFrame into
`species` GeoDataFrame (1) by using `gpd.sjoin()` -function

In [ ]:
# Make a spatial join
joined = gpd.sjoin(species, lulc, how="inner", predicate="within")

# Let's check the result
display(joined.head(5))

Well done. Now we have performed a successful spatial join where we got
two new columns into our `join` GeoDataFrame, i.e. `index_right` that
tells the index of the matching polygon in the `lulc` layer and
`species`.

We could save this layer into a file and review it in QGIS:

```python
# Output path
outfp = "../files/data/L3/landuse_per_species.gpkg"

# Save to disk
joined.to_file(outfp, driver="GPKG", layer='landuse_per_species')
```

Do the results make sense? Let's evaluate this a bit by grouping and
querying the resulting `join` for largest landuse type and species types
combinations:

In [ ]:
# how many times each species is represented in the data
display(joined['NIMI'].value_counts())

In [ ]:
# how many times each landuse type is represented in the data
display(joined['LABEL2'].value_counts())

Those values by themselves only give a cursorily view of the data. Let's make some 
meaningful aggregations.

In [ ]:
# initialise empty list
data_list = []

for species_id, species_group in joined.groupby('NIMI'):
    lulc_count = species_group['LABEL2'].value_counts()
    top = lulc_count.head(1)
    # add info to list
    data_row_as_dict = {'species_id':species_id,
                        'all_sights': len(species_group),
                        'top_lulc': top.index[0],
                        'sights_in_top': top[0] }
    data_list.append(data_row_as_dict)

# Creates DataFrame from now filled list with data items.
top_sights = pd.DataFrame(data_list)

# Print the data
top_sights.sort_values(by=['all_sights','sights_in_top'], ascending=False).head(5)

The "valge-toonekurk" (white stork) is the most sighted bird species in the data, and 15 out of 20 sights are in 
the "Heterogeneous agricultural areas" landuse type.

**Download the notebook:**

[file:geopandas-basics.ipynb](../files/data/L3/spatial-join.ipynb)

**Launch in the web/MyBinder:**

[![](../files/img/colab-badge.svg)](https://colab.research.google.com/github/LandscapeGeoinformatics/geopy-2023/blob/main/L3/spatial-join.ipynb){target="blank"}

[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/allixender/testgeo2020b/master?filepath=L3%2Fspatial-join.ipynb)